<a href="https://colab.research.google.com/github/junghoum/Hello-world/blob/main/n214-logistic-regression/n214a-logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 4*

# 📝 Assignment
---

# Logistic Regression

이번에는 [**Cardiovascular Disease dataset**](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) 데이터셋을 사용해 봅시다. 데이터에 대한 설명을 캐글 페이지에서 잘 살펴보시고 과제를 진행해 주세요!

### 1) 데이터를 전처리 하겠습니다.
- 데이터를 불러오세요.(read_csv 사용)
- id 칼럼은 drop하세요.
- missing value를 확인하세요.
- 중복된 샘플이 있는지 확인하세요.
- **중복된 샘플을 제거하고 남은 샘플의 수를 제출하세요.**

In [88]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')

In [89]:
df = df.drop(columns=['id'], axis=1)

In [90]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [91]:
df.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [92]:
df.duplicated().sum()

24

In [93]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [94]:
df.sample

<bound method NDFrame.sample of          age  gender  height  weight  ap_hi  ...  gluc  smoke  alco  active  cardio
0      18393       2     168    62.0    110  ...     1      0     0       1       0
1      20228       1     156    85.0    140  ...     1      0     0       1       1
2      18857       1     165    64.0    130  ...     1      0     0       0       1
3      17623       2     169    82.0    150  ...     1      0     0       1       1
4      17474       1     156    56.0    100  ...     1      0     0       0       0
...      ...     ...     ...     ...    ...  ...   ...    ...   ...     ...     ...
69995  19240       2     168    76.0    120  ...     1      1     0       1       0
69996  22601       1     158   126.0    140  ...     2      0     0       1       1
69997  19066       2     183   105.0    180  ...     1      0     1       0       1
69998  22431       1     163    72.0    135  ...     2      0     0       0       1
69999  20540       1     170    72.0    120 

In [95]:
len(df)

69976

### 2) 다운로드 한 훈련 데이터를 훈련/검증/테스트 세트로 나누어 주세요.
- 타겟 특성을 확인하고 타겟 변수를 나누어 줍니다.
- 먼저, 훈련 세트와 테스트 세트로 나누어 줍니다(train_test_split 사용, train/test = 80/20, random_state=2)
- 다시, 훈련 세트를 훈련 세트와 검증 세트로 나누어 줍니다(train_test_split 사용, train/validation = 80/20, random_state=2)
- **검증 세트(X_val)의 개수를 제출하세요.**

In [96]:
from sklearn.model_selection import train_test_split

# split dataframe to train / test set
X_train, X_test = train_test_split(df, test_size = 0.2, train_size=0.8, random_state=2)
# split train set to train / validation set
X_train, X_val = train_test_split(X_train, test_size = 0.2, train_size=0.8, random_state=2)

# save target vairable for each set
y_train = X_train['cardio']
y_val = X_val['cardio']
y_test = X_test['cardio']

# drop target variable from sets
X_train = X_train.drop(columns='cardio')
X_val = X_val.drop(columns='cardio')
X_test = X_test.drop(columns='cardio')

len(X_val)

11196

### 3) 사이킷런의 LogisticRegression을 사용해 아래 기준에 맞춰 모델을 만들어 보세요.
- 모델을 만들기 전에 기준모델을 설정합니다.
- 하이퍼파라미터 max_iter=1000을 사용합니다.
- **LogisticRegression 모델을 학습하고 검증세트 정확도를 제출하세요.**


In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# check frequency of values in Target variable in Train Set
y_train.value_counts(normalize=True)
# create baseline model using Train set
y_pred = [y_train.mode()] * len(y_train) 
# check Accuracy of Base Model
print("training accuracy: ", accuracy_score(y_train, y_pred))

model = LogisticRegression(max_iter=1000)

# fit
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print(accuracy_score(y_val, y_pred))

print(model.score(X_val, y_val))

training accuracy:  0.501697034655234
0.7073061807788495
0.7073061807788495


### 4) 데이터에서 범주형 성질을 지닌 특성을 찾아 원핫인코딩 하세요.
- category_encoders를 사용해서 'gender' 특성만 원핫인코딩 하세요.
- LogisticRegression 모델을 인코딩된 데이터로 다시 학습합니다. (하이퍼파라미터 max_iter=1000 동일하게 적용)
- **검증세트 정확도를 제출하세요.**

In [104]:
encoder = OneHotEncoder(use_cat_names=True, cols='gender')
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_train_encoded.head()

,age,gender_1.0,gender_2.0,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
221,17591,1,0,157,70.0,160,100,3,1,0,0,0
59211,19677,0,1,174,80.0,150,80,1,1,0,0,1
26654,23504,1,0,158,76.0,140,90,3,3,0,0,1
33421,18036,0,1,173,128.0,160,110,3,1,0,1,0
12046,20225,0,1,176,109.0,120,80,1,1,1,1,1


In [129]:
new_model = LogisticRegression(max_iter=1000)
new_model.fit(X_train_encoded,y_train)
y_pred = new_model.predict(X_val_encoded)

In [134]:
new_model.score(X_val_encoded, y_val)

0.6944444444444444

(2.690834357157119e-16, 1.0)

ValueError: ignored

### 5) 학습한 모델을 검증세트에서 최적화 시킨 후, 테스트세트에서 딱 한 번 정확도를 확인해 보세요.
- **테스트 세트 정확도를 제출하세요.(0.712 이상)**

In [138]:
inputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_val_imputed = imputer.transform(X_val_encoded)

In [139]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

In [140]:
X_train_scaled.T[0].mean(), X_train_scaled.T[0].std()

(-1.3390900075507426e-16, 1.0)

In [142]:
new_model_2 = LogisticRegression(random_state=1)
new_model_2.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [143]:
y_pred_2 = new_model_2.predict(X_val_scaled)
accuracy_score(y_val, y_pred_2)

0.723383351196856

In [144]:
new_model_2.score(X_val_scaled, y_val)

0.723383351196856

## 🔥 도전과제:

### 공지로 안내드리는 Github - Discussion 페이지에 올려보세요:)

### 6) EDA를 추가적으로 수행하고 아래 질문에 대한 본인만의 근거와 수행결과를 리포트해 보시고 서로 논해 보세요. 
- 새로운 특성을 만들 수 있다면, 어떤 특성공학을 해 볼 수 있을까요? 
- 아웃라이어가 있는 특성이 있다면, 어떤 기준으로 제거할 수 있을까요? 
- [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) 문서를 보면 여러가지 종류의 Scalar가 있는데, 각 Scalar는 각각 어떤 상황에 적용하면 좋을까요? 본 데이터에서는 어떤 scalar를 적용해볼 수 있을까요? 
- 모델 성능을 높이기 위해 어떤 튜닝을 추가해 볼 수 있을까요? 
